In [12]:
import torch
from models import BaseVAE
from torch import nn
from torch.nn import functional as F
from models.types_ import *


class VanillaVAE(BaseVAE):

    def __init__(self,
                 in_channels: int,
                 in_size: int, 
                 latent_dim: int,
                 hidden_dims: List = None,
                 **kwargs) -> None:
        super(VanillaVAE, self).__init__()

        self.latent_dim = latent_dim
        self.nb_last_channels = hidden_dims[-1]
        self.out_channels = in_channels
        

        modules = []
        if hidden_dims is None:
            hidden_dims = [32, 64, 128, 256, 512]
            div = 2**5  # Model reduces size by this factor           
        else:
            div = 2**len(hidden_dims)   # Model reduces size by this factor   
            
        # Make sure input dimension is usable     
        if in_size%div==0:
            self.smallest_size = int(in_size/div)
        else: raise ValueError('Input size not compatible with number of hidden layers.')

            

        # Build Encoder
        for h_dim in hidden_dims:
            modules.append(
                nn.Sequential(
                    nn.Conv2d(in_channels, out_channels=h_dim,
                              kernel_size= 3, stride= 2, padding  = 1),
                    nn.BatchNorm2d(h_dim),
                    nn.LeakyReLU())
            )
            in_channels = h_dim

        self.encoder = nn.Sequential(*modules)
        self.fc_mu = nn.Linear(hidden_dims[-1]*self.smallest_size**2, latent_dim)
        self.fc_var = nn.Linear(hidden_dims[-1]*self.smallest_size**2, latent_dim)


        # Build Decoder
        modules = []

        self.decoder_input = nn.Linear(latent_dim, hidden_dims[-1] * self.smallest_size**2)

        hidden_dims.reverse()

        for i in range(len(hidden_dims) - 1):
            modules.append(
                nn.Sequential(
                    nn.ConvTranspose2d(hidden_dims[i],
                                       hidden_dims[i + 1],
                                       kernel_size=3,
                                       stride = 2,
                                       padding=1,
                                       output_padding=1),
                    nn.BatchNorm2d(hidden_dims[i + 1]),
                    nn.LeakyReLU())
            )



        self.decoder = nn.Sequential(*modules)

        self.final_layer = nn.Sequential(
                            nn.ConvTranspose2d(hidden_dims[-1],
                                               hidden_dims[-1],
                                               kernel_size=3,
                                               stride=2,
                                               padding=1,
                                               output_padding=1),
                            nn.BatchNorm2d(hidden_dims[-1]),
                            nn.LeakyReLU(),
                            nn.Conv2d(hidden_dims[-1], out_channels= self.out_channels,
                                      kernel_size= 3, padding= 1),
                            nn.Tanh())

    def encode(self, input: Tensor) -> List[Tensor]:
        """
        Encodes the input by passing through the encoder network
        and returns the latent codes.
        :param input: (Tensor) Input tensor to encoder [N x C x H x W]
        :return: (Tensor) List of latent codes
        """
        result = self.encoder(input)
        result = torch.flatten(result, start_dim=1)

        # Split the result into mu and var components
        # of the latent Gaussian distribution
        mu = self.fc_mu(result)
        log_var = self.fc_var(result)

        return [mu, log_var]

    def decode(self, z: Tensor) -> Tensor:
        """
        Maps the given latent codes
        onto the image space.
        :param z: (Tensor) [B x D]
        :return: (Tensor) [B x C x H x W]
        """
        result = self.decoder_input(z)
        result = result.view(-1, self.nb_last_channels, self.smallest_size, self.smallest_size)
        result = self.decoder(result)
        result = self.final_layer(result)
        return result

    def reparameterize(self, mu: Tensor, logvar: Tensor) -> Tensor:
        """
        Reparameterization trick to sample from N(mu, var) from
        N(0,1).
        :param mu: (Tensor) Mean of the latent Gaussian [B x D]
        :param logvar: (Tensor) Standard deviation of the latent Gaussian [B x D]
        :return: (Tensor) [B x D]
        """
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return eps * std + mu

    def forward(self, input: Tensor, **kwargs) -> List[Tensor]:
        mu, log_var = self.encode(input)
        z = self.reparameterize(mu, log_var)
        return  [self.decode(z), input, mu, log_var]

    def loss_function(self,
                      *args,
                      **kwargs) -> dict:
        """
        Computes the VAE loss function.
        KL(N(\mu, \sigma), N(0, 1)) = \log \frac{1}{\sigma} + \frac{\sigma^2 + \mu^2}{2} - \frac{1}{2}
        :param args:
        :param kwargs:
        :return:
        """
        recons = args[0]
        input = args[1]
        mu = args[2]
        log_var = args[3]

        kld_weight = kwargs['M_N'] # Account for the minibatch samples from the dataset
        recons_loss =F.mse_loss(recons, input)


        kld_loss = torch.mean(-0.5 * torch.sum(1 + log_var - mu ** 2 - log_var.exp(), dim = 1), dim = 0)

        loss = recons_loss + kld_weight * kld_loss
        return {'loss': loss, 'Reconstruction_Loss':recons_loss, 'KLD':-kld_loss}

    def sample(self,
               num_samples:int,
               current_device: int, **kwargs) -> Tensor:
        """
        Samples from the latent space and return the corresponding
        image space map.
        :param num_samples: (Int) Number of samples
        :param current_device: (Int) Device to run the model
        :return: (Tensor)
        """
        z = torch.randn(num_samples,
                        self.latent_dim)

        z = z.to(current_device)

        samples = self.decode(z)
        return samples

    def generate(self, x: Tensor, **kwargs) -> Tensor:
        """
        Given an input image x, returns the reconstructed image
        :param x: (Tensor) [B x C x H x W]
        :return: (Tensor) [B x C x H x W]
        """

        return self.forward(x)[0]

In [13]:
from torchvision.utils import save_image

# Achtung: Momentan wird im Trainer nur Reconstruction Loss verwendet
class Trainer(object):
    def __init__(self, model,
                 optimizer, loss_function,
                 loader_train, loader_val,
                 dtype, device, **in_params):
        """
        :param model: PyTorch model of the neural network

        :param optimizer: PyTorch optimizer

        :param print_every: How often should we print the loss during training
        """
        # Create attributes:
        self.device = device
        self.model = model.to(device=self.device)  # move the model parameters to CPU/GPU
        self.optimizer = optimizer
        self.loss_function = loss_function
        self.loader_train = loader_train
        self.loader_val = loader_val
        self.print_every = in_params["print_every"]
        self.dtype = dtype
        self.batch_size = in_params["batch_size"]
        self.input_size = in_params["input_size"]
        self.path = in_params["path"]


    def train_model(self, epoch):
        """
        - epoch: An integer giving the epoch
        """
        train_loss = 0
        self.model.train()  # put model to training mode
        for t, (input,_) in enumerate(self.loader_train):
            input = input.to(device=self.device, dtype=self.dtype)  # move to device, e.g. GPU
            
            # do a step in training
            args = self.model(input)
            loss = self.loss_function(*args,**{'M_N':self.batch_size/len(self.loader_train)})['Reconstruction_Loss']
            self.optimizer.zero_grad()
            loss.backward() 
            train_loss += loss.item() # accumulate for average loss
            self.optimizer.step()

            # print loss
            if t % self.print_every == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, t * len(args[1]), len(self.loader_train.dataset),
                    100. * t / len(self.loader_train),
                    loss.item() / len(args[1])))
        # print average loss
        print('====> Epoch: {} Average loss: {:.6f}'.format(
              epoch, train_loss / len(self.loader_train.dataset)))

    def test_model(self, epoch):
        self.model.eval() # Put model to evaluation mode
        test_loss = 0.

        with torch.no_grad():
            # During validation, we accumulate these values across the whole dataset and then average at the end:
            for i, (input,_) in enumerate(self.loader_val):
                input = input.to(device=self.device, dtype=self.dtype)  # move to device, e.g. GPU
       
                # compute loss and accumulate
                args = self.model(input)
                test_loss += self.loss_function(*args,**{'M_N':self.batch_size/len(self.loader_val)})['Reconstruction_Loss'].item()
                if i == 0:
                    n = min(args[1].size(0), 8)
                    comparison = torch.cat([args[1][:n],
                                          args[0].view(self.batch_size, self.model.out_channels, self.input_size, self.input_size)[:n]])
                    save_image(comparison.cpu(),
                             self.path + '/reconstruction_' + str(epoch) + '.png', nrow=n)

        # print average loss
        test_loss /= len(self.loader_val.dataset)
        print('====> Test set loss: {:.6f}'.format(test_loss))
        
    def train_and_test(self, epochs):
        for e in range(1,epochs+1):
            self.train_model(e)
            self.test_model(e)
            with torch.no_grad():
                sample = self.model.sample(64,device)
                save_image(sample.view(64, self.model.out_channels, self.input_size, self.input_size),
                           self.path + '/sample_' + str(e) + '.png')
        

In [4]:
from torchvision import datasets, transforms

in_params = {"batch_size": 32,
        "epochs": 150,
        "no_cuda": False,
        "seed": 1,
        "print_every": 10,
        "input_size": 64,
        "path": 'results'
        }
in_params["cuda"] = not in_params["no_cuda"] and torch.cuda.is_available()
torch.manual_seed(in_params["seed"])

device = torch.device("cuda" if in_params["cuda"] else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if in_params["cuda"] else {}


model = VanillaVAE(in_channels=3,
                   in_size=in_params["input_size"],
                   latent_dim=250,
                   hidden_dims=[32, 64, 128, 256])

# transformations of input images before feeding into nn
transformations = transforms.Compose([
    transforms.Resize(128),
    transforms.RandomCrop(in_params["input_size"]),
    transforms.ToTensor()])

#load data into DataLoader
train_dataset = datasets.ImageFolder('data_to_try/full_data/training', 
                                 transform=transformations
                                   )
test_dataset = datasets.ImageFolder('data_to_try/full_data/test', 
                                 transform=transformations
                                   )



train_loader = torch.utils.data.DataLoader(train_dataset,
                                           batch_size=in_params["batch_size"], 
                                           shuffle=True,
                                           drop_last=True,
                                           **kwargs
                                           )
test_loader = torch.utils.data.DataLoader(test_dataset,
                                          batch_size=in_params["batch_size"], 
                                          shuffle=True,
                                          drop_last=True,
                                          **kwargs
                                          )

# Build the optimizer:
params = model.parameters()
learning_rate = 1e-4
optimizer = torch.optim.AdamW(params, lr=learning_rate)


# Build the trainer with the Soresen-Dice loss you implemented:
trainer = Trainer(model, optimizer, model.loss_function,
        train_loader, test_loader, torch.float32, device,**in_params )

# Start training:
trainer.train_and_test(in_params["epochs"])

Train Epoch: 1 [0/1620 (0%)]	Loss: 0.009494
Train Epoch: 1 [320/1620 (20%)]	Loss: 0.005811
Train Epoch: 1 [640/1620 (40%)]	Loss: 0.004198
Train Epoch: 1 [960/1620 (60%)]	Loss: 0.003090
Train Epoch: 1 [1280/1620 (80%)]	Loss: 0.002166
====> Epoch: 1 Average loss: 0.0043
====> Test set loss: 0.0033
Train Epoch: 2 [0/1620 (0%)]	Loss: 0.001847
Train Epoch: 2 [320/1620 (20%)]	Loss: 0.001516
Train Epoch: 2 [640/1620 (40%)]	Loss: 0.001108
Train Epoch: 2 [960/1620 (60%)]	Loss: 0.001001
Train Epoch: 2 [1280/1620 (80%)]	Loss: 0.000965
====> Epoch: 2 Average loss: 0.0012
====> Test set loss: 0.0013
Train Epoch: 3 [0/1620 (0%)]	Loss: 0.000860
Train Epoch: 3 [320/1620 (20%)]	Loss: 0.000857
Train Epoch: 3 [640/1620 (40%)]	Loss: 0.000820
Train Epoch: 3 [960/1620 (60%)]	Loss: 0.000839
Train Epoch: 3 [1280/1620 (80%)]	Loss: 0.000800
====> Epoch: 3 Average loss: 0.0008
====> Test set loss: 0.0010
Train Epoch: 4 [0/1620 (0%)]	Loss: 0.000720
Train Epoch: 4 [320/1620 (20%)]	Loss: 0.000775
Train Epoch: 4 [64

Train Epoch: 28 [640/1620 (40%)]	Loss: 0.000188
Train Epoch: 28 [960/1620 (60%)]	Loss: 0.000233
Train Epoch: 28 [1280/1620 (80%)]	Loss: 0.000216
====> Epoch: 28 Average loss: 0.0002
====> Test set loss: 0.0004
Train Epoch: 29 [0/1620 (0%)]	Loss: 0.000212
Train Epoch: 29 [320/1620 (20%)]	Loss: 0.000236
Train Epoch: 29 [640/1620 (40%)]	Loss: 0.000206
Train Epoch: 29 [960/1620 (60%)]	Loss: 0.000218
Train Epoch: 29 [1280/1620 (80%)]	Loss: 0.000247
====> Epoch: 29 Average loss: 0.0002
====> Test set loss: 0.0004
Train Epoch: 30 [0/1620 (0%)]	Loss: 0.000275
Train Epoch: 30 [320/1620 (20%)]	Loss: 0.000227
Train Epoch: 30 [640/1620 (40%)]	Loss: 0.000197
Train Epoch: 30 [960/1620 (60%)]	Loss: 0.000228
Train Epoch: 30 [1280/1620 (80%)]	Loss: 0.000244
====> Epoch: 30 Average loss: 0.0002
====> Test set loss: 0.0004
Train Epoch: 31 [0/1620 (0%)]	Loss: 0.000222
Train Epoch: 31 [320/1620 (20%)]	Loss: 0.000303
Train Epoch: 31 [640/1620 (40%)]	Loss: 0.000204
Train Epoch: 31 [960/1620 (60%)]	Loss: 0.00

Train Epoch: 55 [960/1620 (60%)]	Loss: 0.000142
Train Epoch: 55 [1280/1620 (80%)]	Loss: 0.000149
====> Epoch: 55 Average loss: 0.0002
====> Test set loss: 0.0003
Train Epoch: 56 [0/1620 (0%)]	Loss: 0.000151
Train Epoch: 56 [320/1620 (20%)]	Loss: 0.000170
Train Epoch: 56 [640/1620 (40%)]	Loss: 0.000160
Train Epoch: 56 [960/1620 (60%)]	Loss: 0.000188
Train Epoch: 56 [1280/1620 (80%)]	Loss: 0.000238
====> Epoch: 56 Average loss: 0.0002
====> Test set loss: 0.0003
Train Epoch: 57 [0/1620 (0%)]	Loss: 0.000185
Train Epoch: 57 [320/1620 (20%)]	Loss: 0.000185
Train Epoch: 57 [640/1620 (40%)]	Loss: 0.000168
Train Epoch: 57 [960/1620 (60%)]	Loss: 0.000152
Train Epoch: 57 [1280/1620 (80%)]	Loss: 0.000165
====> Epoch: 57 Average loss: 0.0002
====> Test set loss: 0.0003
Train Epoch: 58 [0/1620 (0%)]	Loss: 0.000164
Train Epoch: 58 [320/1620 (20%)]	Loss: 0.000168
Train Epoch: 58 [640/1620 (40%)]	Loss: 0.000189
Train Epoch: 58 [960/1620 (60%)]	Loss: 0.000171
Train Epoch: 58 [1280/1620 (80%)]	Loss: 0.0

Train Epoch: 82 [1280/1620 (80%)]	Loss: 0.000137
====> Epoch: 82 Average loss: 0.0002
====> Test set loss: 0.0003
Train Epoch: 83 [0/1620 (0%)]	Loss: 0.000196
Train Epoch: 83 [320/1620 (20%)]	Loss: 0.000172
Train Epoch: 83 [640/1620 (40%)]	Loss: 0.000141
Train Epoch: 83 [960/1620 (60%)]	Loss: 0.000127
Train Epoch: 83 [1280/1620 (80%)]	Loss: 0.000148
====> Epoch: 83 Average loss: 0.0002
====> Test set loss: 0.0003
Train Epoch: 84 [0/1620 (0%)]	Loss: 0.000139
Train Epoch: 84 [320/1620 (20%)]	Loss: 0.000144
Train Epoch: 84 [640/1620 (40%)]	Loss: 0.000133
Train Epoch: 84 [960/1620 (60%)]	Loss: 0.000190
Train Epoch: 84 [1280/1620 (80%)]	Loss: 0.000162
====> Epoch: 84 Average loss: 0.0002
====> Test set loss: 0.0003
Train Epoch: 85 [0/1620 (0%)]	Loss: 0.000169
Train Epoch: 85 [320/1620 (20%)]	Loss: 0.000154
Train Epoch: 85 [640/1620 (40%)]	Loss: 0.000151
Train Epoch: 85 [960/1620 (60%)]	Loss: 0.000132
Train Epoch: 85 [1280/1620 (80%)]	Loss: 0.000162
====> Epoch: 85 Average loss: 0.0002
====>

Train Epoch: 109 [1280/1620 (80%)]	Loss: 0.000150
====> Epoch: 109 Average loss: 0.0001
====> Test set loss: 0.0003
Train Epoch: 110 [0/1620 (0%)]	Loss: 0.000143
Train Epoch: 110 [320/1620 (20%)]	Loss: 0.000125
Train Epoch: 110 [640/1620 (40%)]	Loss: 0.000159
Train Epoch: 110 [960/1620 (60%)]	Loss: 0.000153
Train Epoch: 110 [1280/1620 (80%)]	Loss: 0.000121
====> Epoch: 110 Average loss: 0.0001
====> Test set loss: 0.0003
Train Epoch: 111 [0/1620 (0%)]	Loss: 0.000144
Train Epoch: 111 [320/1620 (20%)]	Loss: 0.000164
Train Epoch: 111 [640/1620 (40%)]	Loss: 0.000117
Train Epoch: 111 [960/1620 (60%)]	Loss: 0.000157
Train Epoch: 111 [1280/1620 (80%)]	Loss: 0.000134
====> Epoch: 111 Average loss: 0.0001
====> Test set loss: 0.0003
Train Epoch: 112 [0/1620 (0%)]	Loss: 0.000143
Train Epoch: 112 [320/1620 (20%)]	Loss: 0.000165
Train Epoch: 112 [640/1620 (40%)]	Loss: 0.000126
Train Epoch: 112 [960/1620 (60%)]	Loss: 0.000131
Train Epoch: 112 [1280/1620 (80%)]	Loss: 0.000117
====> Epoch: 112 Averag

Train Epoch: 136 [320/1620 (20%)]	Loss: 0.000139
Train Epoch: 136 [640/1620 (40%)]	Loss: 0.000166
Train Epoch: 136 [960/1620 (60%)]	Loss: 0.000127
Train Epoch: 136 [1280/1620 (80%)]	Loss: 0.000184
====> Epoch: 136 Average loss: 0.0001
====> Test set loss: 0.0003
Train Epoch: 137 [0/1620 (0%)]	Loss: 0.000126
Train Epoch: 137 [320/1620 (20%)]	Loss: 0.000115
Train Epoch: 137 [640/1620 (40%)]	Loss: 0.000119
Train Epoch: 137 [960/1620 (60%)]	Loss: 0.000126
Train Epoch: 137 [1280/1620 (80%)]	Loss: 0.000138
====> Epoch: 137 Average loss: 0.0001
====> Test set loss: 0.0003
Train Epoch: 138 [0/1620 (0%)]	Loss: 0.000115
Train Epoch: 138 [320/1620 (20%)]	Loss: 0.000133
Train Epoch: 138 [640/1620 (40%)]	Loss: 0.000143
Train Epoch: 138 [960/1620 (60%)]	Loss: 0.000134
Train Epoch: 138 [1280/1620 (80%)]	Loss: 0.000133
====> Epoch: 138 Average loss: 0.0001
====> Test set loss: 0.0002
Train Epoch: 139 [0/1620 (0%)]	Loss: 0.000160
Train Epoch: 139 [320/1620 (20%)]	Loss: 0.000124
Train Epoch: 139 [640/16

In [14]:
# For MNIST dataset

in_params = {"batch_size": 128,
        "epochs": 5,
        "no_cuda": False,
        "seed": 1,
        "print_every": 100,
        "input_size": 28,
        "path": 'results_mnist'
        }
in_params["cuda"] = not in_params["no_cuda"] and torch.cuda.is_available()
torch.manual_seed(in_params["seed"])

device = torch.device("cuda" if in_params["cuda"] else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if in_params["cuda"] else {}


model = VanillaVAE(in_channels=1,
                   in_size=in_params["input_size"],
                   latent_dim=100,
                   hidden_dims=[64, 128])


train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.ToTensor()),
    batch_size=in_params["batch_size"], shuffle=True,drop_last=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.ToTensor()),
    batch_size=in_params["batch_size"], shuffle=True,drop_last=True, **kwargs)

# Build the optimizer:
params = model.parameters()
learning_rate = 1e-4
optimizer = torch.optim.AdamW(params, lr=learning_rate)


# Build the trainer with the Soresen-Dice loss you implemented:
trainer = Trainer(model, optimizer, model.loss_function,
        train_loader, test_loader, torch.float32, device,**in_params )

# Start training:
trainer.train_and_test(in_params["epochs"])

Train Epoch: 1 [0/60000 (0%)]	Loss: 0.004352
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.000308
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.000183
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.000132
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.000110
====> Epoch: 1 Average loss: 0.000307
====> Test set loss: 0.000094
Train Epoch: 2 [0/60000 (0%)]	Loss: 0.000094
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.000085
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.000072
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.000063
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.000063
====> Epoch: 2 Average loss: 0.000074
====> Test set loss: 0.000055
Train Epoch: 3 [0/60000 (0%)]	Loss: 0.000054
Train Epoch: 3 [12800/60000 (21%)]	Loss: 0.000054
Train Epoch: 3 [25600/60000 (43%)]	Loss: 0.000045
Train Epoch: 3 [38400/60000 (64%)]	Loss: 0.000043
Train Epoch: 3 [51200/60000 (85%)]	Loss: 0.000039
====> Epoch: 3 Average loss: 0.000047
====> Test set loss: 0.000038
Train Epoch: 4 [0/60000 (0%)]	Loss: 0.000041
Train Epoch: 4 [